In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

from tqdm import tqdm_notebook as tqdm
from copy import copy, deepcopy
from utils import data_generator, pre_process, process_output

In [2]:
IMAGE_FOLDER = "patch_images/train"

image_paths = [IMAGE_FOLDER + "/" + name for _,_,files in os.walk(IMAGE_FOLDER) for name in files]
gen = data_generator(image_paths)

In [3]:
#123456789
n_bins = 313
class Model(tf.keras.Model):
    def __init__(self):
        super(Model, self).__init__()
        # (batch_size, 256, 256, 1) --> (batch_size, 128, 128, 64)
        self.conv1 = tf.keras.layers.Conv2D(filters=64, kernel_size=3,
                                            strides=(2, 2),
                                            padding='same',
                                            activation='relu',
                                            input_shape=(32, 32, 1))
        self.bn1 = tf.keras.layers.BatchNormalization()
        
        # (batch_size, 128, 128, 64) --> (batch_size, 64, 64, 128)
        self.conv2 = tf.keras.layers.Conv2D(filters=128, kernel_size=3,
                                            strides=(2, 2),
                                            padding='same',
                                            activation='relu')
        self.bn2 = tf.keras.layers.BatchNormalization()

        
        # (batch_size, 64, 64, 128)  --> (batch_size, 32, 32, 256)
        self.conv3 = tf.keras.layers.Conv2D(filters=256, kernel_size=3,
                                            strides=(2, 2),
                                            padding='same',
                                            activation='relu')
        self.bn3 = tf.keras.layers.BatchNormalization()
        
        # (batch_size, 32, 32, 256) --> (batch_size, 32, 32, 512)
        self.conv4 = tf.keras.layers.Conv2D(filters=512, kernel_size=3,
                                            strides=(1, 1),
                                            dilation_rate=(1, 1),
                                            padding='same',
                                            activation='relu'),
        self.bn4 = tf.keras.layers.BatchNormalization()
        
        # (batch_size, 32, 32, 512) --> (batch_size, 32, 32, 512)
        self.deconv1 = tf.keras.layers.Conv2DTranspose(filters=512, kernel_size=3,
                                                       strides=(1, 1),
                                                       dilation_rate=(2, 2),
                                                       padding='same',
                                                       activation='relu')
        self.bn5 = tf.keras.layers.BatchNormalization()
        
        # (batch_size, 32, 32, 512) --> (batch_size, 32, 32, 512)
        self.deconv2 = tf.keras.layers.Conv2DTranspose(filters=512, kernel_size=3,
                                                       dilation_rate=(2, 2),
                                                       padding='same',
                                                       activation='relu')
        self.bn6 = tf.keras.layers.BatchNormalization()
        
        # (batch_size, 32, 32, 512) --> (batch_size, 32, 32, 512)
        self.deconv3 = tf.keras.layers.Conv2DTranspose(filters=512, kernel_size=3,
                                                       dilation_rate=(1, 1),
                                                       padding='same',
                                                       activation='relu')
        self.bn7 = tf.keras.layers.BatchNormalization()
        
        # (batch_size, 64, 64, 256) --> (batch_size, 64, 64, n_bins)
        self.deconv_a = tf.keras.layers.Conv2DTranspose(filters=n_bins, kernel_size=1,
                                                        activation='softmax',
                                                        dilation_rate=(1, 1))
        self.deconv_b = tf.keras.layers.Conv2DTranspose(filters=n_bins, kernel_size=1,
                                                        activation='softmax',
                                                        dilation_rate=(1, 1))
        
        self.seq_layers = [self.conv1, self.bn1,
                           self.conv2, self.bn2,
                           self.conv3, self.bn3,
                           self.conv4, self.bn4,
                           self.deconv1, self.bn5,
                           self.deconv2, self.bn6,
                           self.deconv3, self.bn7]
        
    def call(self, inputs):
        x = inputs
        for layer in self.seq_layers:
            x = layer(x)
        probs_a = self.deconv_a(x)
        probs_b = self.deconv_b(x)
        return probs_a, probs_b
        
model = Model()

In [4]:
data = tf.data.Dataset.from_generator(lambda: data_generator(image_paths),
                                          output_types=(np.float32, np.int64))
data = data.shuffle(1000).batch(32)

W0429 14:07:09.013457 4327654848 deprecation.py:323] From /Users/reniel96/anaconda3/lib/python3.5/site-packages/tensorflow/python/data/ops/dataset_ops.py:410: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


In [5]:
data

<DatasetV1Adapter shapes: (<unknown>, <unknown>), types: (tf.float32, tf.int64)>

In [6]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy_a = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy_a')
train_accuracy_b = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy_b')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy_a = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy_a')
test_accuracy_b = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy_b')
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

@tf.function
def train_step(image, labels_a, labels_b):
    with tf.GradientTape() as tape:
        probs_a, probs_b = model(image)
        loss_a = loss_object(labels_a, probs_a)
        loss_b = loss_object(labels_b, probs_b)
        loss = loss_a + loss_b
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy_a(labels_a, probs_a)
    train_accuracy_b(labels_b, probs_b)
    
@tf.function
def test_step(image, labels_a, labels_b):
    probs_a, probs_b = model(image)
    t_loss_a = loss_object(labels_a, probs_a)
    t_loss_b = loss_object(labels_b, probs_b)
    t_loss = t_loss_a + t_loss_b

    test_loss(t_loss)
    test_accuracy_a(labels_a, probs_a)
    test_accuracy_b(labels_b, probs_b)

In [7]:
EPOCHS = 5

for epoch in range(EPOCHS):
    for index, (batch_luminance, batch_ab) in tqdm(enumerate(data), total=1000//32):
        train_step(batch_luminance, batch_ab[:, 0, :, : , :], batch_ab[:, 1, :, : , :])
        if index % 100 == 0:
            template = 'Epoch {}, Step {}, Loss: {}, Accuracy a: {}, Accuracy b: {}'
            print(template.format(epoch+1,
                                  index,
                                  round(float(train_loss.result()), 3),
                                  round(float(train_accuracy_a.result()) * 100, 2),
                                  round(float(train_accuracy_b.result()) * 100, 2)))

    template = 'Epoch {}, Loss: {}, Accuracy a: {}, Accuracy b: {}'
    print(template.format(epoch + 1,
                          round(float(train_loss.result()), 3),
                          round(float(train_accuracy_a.result()) * 100, 2),
                          round(float(train_accuracy_b.result()) * 100, 2)))

InvalidArgumentError: TypeError: 'NoneType' object is not subscriptable
Traceback (most recent call last):

  File "/Users/reniel96/anaconda3/lib/python3.5/site-packages/tensorflow/python/ops/script_ops.py", line 207, in __call__
    ret = func(*args)

  File "/Users/reniel96/anaconda3/lib/python3.5/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 430, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/Users/reniel96/Desktop/Berkeley/Spring Semester/IEOR 242/IEOR242_deep_colorization/utils.py", line 77, in data_generator
    rgb_image = bgr_image[:, :, ::-1]

TypeError: 'NoneType' object is not subscriptable


	 [[{{node PyFunc}}]] [Op:IteratorGetNextSync]